In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/issues-27-9-2024/issues_test.csv
/kaggle/input/issues-27-9-2024/issues_train.csv


In [2]:
train_path = '/kaggle/input/issues-27-9-2024/issues_train.csv'
test_path =  '/kaggle/input/issues-27-9-2024/issues_test.csv'

In [3]:
import pandas as pd
import numpy as np

def add_collum(src_name, col_name, frame):
  frame[col_name] = np.where(frame[src_name] == col_name, 1, 0)
  return frame

def preprocessing(csv_name):
  df = pd.read_csv(csv_name)
  for item in df.repo.unique():
    df = add_collum('repo', item, df)
  for item in df.label.unique():
    df = add_collum('label', item, df)
  df['data'] = df.title + ' ' + df.body

  ndf = df.drop(['title', 'body', 'repo', 'label'], axis=1)

  ndf['target_list'] = ndf[['bug', 'feature', 'question',
                          'facebook/react', 'tensorflow/tensorflow',
                          'microsoft/vscode', 'bitcoin/bitcoin',
                          'opencv/opencv']].values.tolist()
  df2 = ndf.drop(['bug', 'feature', 'question',
                          'facebook/react', 'tensorflow/tensorflow',
                          'microsoft/vscode', 'bitcoin/bitcoin',
                          'opencv/opencv'], axis=1)
  return df2

In [4]:
train_df = preprocessing(train_path)
test_df = preprocessing(test_path)
train_df, test_df

(               created_at                                               data  \
 0     2023-08-26 06:33:37  [DevTools Bug] Cannot add node "1" because a n...   
 1     2023-07-28 05:16:12  [DevTools Bug]: Devtools extension build faili...   
 2     2023-07-13 21:58:31  [DevTools Bug]: Deprecated __REACT_DEVTOOLS_GL...   
 3     2023-06-14 02:31:20  [DevTools Bug] Cannot remove node "0" because ...   
 4     2023-06-03 11:29:44  [DevTools Bug] Cannot remove node "103" becaus...   
 ...                   ...                                                ...   
 1495  2022-01-24 10:48:13  core: FP denormals support relates #21046\r\n\...   
 1496  2022-01-20 12:40:55  feature: submodule or a class scope for export...   
 1497  2022-01-15 02:39:22  Reading BigTiff images **Merge with extra: htt...   
 1498  2022-01-14 15:37:53  Add general broadcasting layer Performance det...   
 1499  2022-01-11 16:30:53  Adapt remote inference to operate with NV12 bl...   
 
                    target

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [6]:
MAX_LEN = 250
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 4
LEARNING_RATE = 1e-06 * 5
# https://stackoverflow.com/questions/65082243/dropout-argument-input-position-1-must-be-tensor-not-str-when-using-bert
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', return_dict=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.title = dataframe['data']
        self.targets = self.data.target_list
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


train_size = 0.8
train_dataset = train_df
valid_dataset = test_df
train_dataset = train_dataset.reset_index(drop=True)


#print("FULL Dataset: {}".format(df2.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(valid_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(valid_dataset, tokenizer, MAX_LEN)

TRAIN Dataset: (1500, 3)
TEST Dataset: (1500, 3)


In [8]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)


len(training_loader)

24

In [9]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [10]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 8)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
# model.to(device)
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


len(training_loader)

24

In [12]:

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()


import shutil, sys
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)


#to use as global variables
val_targets=[]
val_outputs=[]


#this is joe method
#https://towardsdatascience.com/how-to-save-and-load-a-model-in-pytorch-with-a-complete-example-c2920e617dee
def train_model(start_epochs,  n_epochs, valid_loss_min_input,
                training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = valid_loss_min_input


  for epoch in range(start_epochs, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # https://stackoverflow.com/questions/65082243/dropout-argument-input-position-1-must-be-tensor-not-str-when-using-bert
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()
    acc = 0
    length = 0
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
           

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
        
      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f} '.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))


  return model


'''start_epochs,  n_epochs, valid_loss_min_input,
                training_loader, validation_loader, model, optimizer,
                checkpoint_path, best_model_path):
'''
checkpoint_path = 'current_checkpoint.pt'
best_model = 'best_model.pt'
trained_model = train_model(1, 30, np.Inf, training_loader, validation_loader, model,
                      optimizer,checkpoint_path,best_model)

############# Epoch 1: Training Start   #############


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.026186 	Average Validation Loss: 0.024093 
Validation loss decreased (inf --> 0.024093).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.023579 	Average Validation Loss: 0.022476 
Validation loss decreased (0.024093 --> 0.022476).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch 3: Validation End     #############
Epoch: 3 	Avg

In [13]:
trained_model = train_model(31, 70, 0.018446, training_loader, validation_loader, model,
                      optimizer,checkpoint_path,best_model)

############# Epoch 31: Training Start   #############
############# Epoch 31: Training End     #############
############# Epoch 31: Validation Start   #############
############# Epoch 31: Validation End     #############
Epoch: 31 	Avgerage Training Loss: 0.005420 	Average Validation Loss: 0.008303 
Validation loss decreased (0.018446 --> 0.008303).  Saving model ...
############# Epoch 31  Done   #############

############# Epoch 32: Training Start   #############
############# Epoch 32: Training End     #############
############# Epoch 32: Validation Start   #############
############# Epoch 32: Validation End     #############
Epoch: 32 	Avgerage Training Loss: 0.005184 	Average Validation Loss: 0.008239 
Validation loss decreased (0.008303 --> 0.008239).  Saving model ...
############# Epoch 32  Done   #############

############# Epoch 33: Training Start   #############
############# Epoch 33: Training End     #############
############# Epoch 33: Validation Start   #########